In [62]:
import pandas as pd
import re
from datetime import datetime
import praw
import pandas as pd
import matplotlib.pyplot as plt



reddit = praw.Reddit(client_id = "QC3sHGjf2WVbx3rZnzJBzw",
                     client_secret = "z7-Qudmgrz4Hj9KXKHT3GYKFiAeg4g",
                     user_agent = "CSE_482_project by Round_Lab_9338")

In [28]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut','Delaware', 'Florida', 'Georgia', 'Hawaii',
          'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
          'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'NewHampshire', 'newjersey', 'NewMexico', 'NewYork',
          'NorthCarolina', 'NorthDakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'RhodeIsland', 'SouthCarolina', 'SouthDakota',
          'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'WestVirginia', 'Wisconsin','Wyoming']
states = list(map(lambda x: x.lower(), states))

# Cities subbreddits are key, and states are the values
cities_to_state = {'birmingham':'alabama',
                   'fairbanks':'alaska',
                   'phoenix': 'arizona', 'tucson': 'arizona',
                   'littlerock':'arkansas', 'fayetteville':'arkansas',
                   'losangeles':'california','sandiego':'california', 'santarosa':'california', 'santabarbara':'california', 'sanfrancisco':'california', 'palmsprings':'california',
                   'coloradosprings':'colorado', 'aurora':'colorado',
                   'hartford':'connecticut',
                   'miami':'flordia', 'orlando': 'flordia', 'jacksonville':'flordia',
                   'atlanta':'georgia', 'augusta':'georgia', 'roswell':'georgia',
                   'honolulu':'hawaii',
                   'boise':'idaho', 'nampa':'idaho',
                   'naperville':'illinois', 'chicago':'illinois',
                   'indianapolis':'indiana', 'bloomington':'indiana',
                   'iowacity':'iowa', 'desmoines':'iowa',
                   'wichita':'kansas', 'kansascity':'kansas',
                   'louisville':'kentucky', 'lexington':'Kentucky',
                   'neworleans':'louisiana', 'shreveport':'louisiana',
                   'frederickmd':'maryland', 'annapolis':'maryland',
                   'boston':'massachusetts', 'salem':'massachusetts',
                   'annarbor':"michigan", "grandrapids":"michigan", 'detroit':"michigan", 'ferndale':'michigan',
                   #couldn't find a subreddit for cities in maine
                   'minneapolis':'minnesota', 'duluth':'minnesota',
                   'jacksonmi':'mississippi', 'tupelo':'mississippi',
                   'bransonmo':'missouri', 'springfield':'missouri',
                   'bozeman':'montana', 'helena':'montana',
                   'omaha':'nebraska', 'kearney':'nebraska',
                   'lasvegas':'nevada', 'sparksnv':'nevada', 'reno':'nevada', 'HendersonNV':'nevada',
                   'portsmouthnh':'newhampshire', 'manchesternh':'newhampshire',
                   'southjersey':'newjersey', 'newark':'newjersey', 'jerseycity':'newjersey',
                   'roswellnm':'newmexico', 'albuquerque':'newmexico',
                   'nyc':'newyork', 'pittsburgh':'newyork', 'buffalo':'newyork', 'columbia':'newyork', 'albany':'newyork',
                   'charlotte':'northcarolina', 'wilmington':'northcarolina', 'greenvillencarolina': 'northcarolina',
                   'fargo':'northdakota', 'bismarck':'northdakota', 'minot':'northdakota',
                   'columbus':'ohio', 'cleveland':'ohio', 'toledo':'ohio', 'cincinnati':'ohio', 'dayton':'ohio', 'akron':'ohio',
                   'tulsa':'oklahoma', 'okc':'oklahoma', #this is oklahoma city subreddit
                   'portland':'oregon', 'portlandor':'oregon', 'bend': 'oregon', 'eugene':'oregon',
                   'philadelphia':'pennsylvania','philly':'pennsylvania', 'pittsburgh':'pennsylvania', 'harrisburg':'pennsylvania', 'erie':'pennsylvania',
                    # rhode island is a city
                   'charleston':'southcarolina', 'myrtlebeach':'southcarolina', 'greenville':'southcarolina',
                   'siouxFalls':'southdakota', 'pierre':'southdakota',
                   'nashville':'tennessee', 'memphis':'tennessee', 'knoxville':'tennessee',
                   'houston':'texas', 'dallas':'texas', 'sanantonio':'texas', 'austin':'texas', 'elpaso':'texas', 'fortworth':'texas', 'lubbock':'texas', 'bedford':'texas',
                   'saltlakecity':'utah', 'provoUtah':'utah', 'stgeorge':'utah',
                   'burlington':'vermont', 'montpelier':'vermont',
                   'langley':'virginia', 'richmondva':'virginia',
                   'seattle':'washington', 'seattlewa':'washington',
                   'morgantownwv':'westvirginia', 'parkersburg':'westvirginia',
                   'milwaukee':'wisconsin', 'madisonwi':'wisconsin', 'appleton':'wisconsin',
                   'cheyenne':'wyoming', 'casper':'wyoming'}


In [17]:
# City to state converter.
# https://github.com/zazencodes/city_to_state_dictionary/blob/master/US%20City%20to%20State%20dictionary.ipynb
saved_tesla = pd.read_csv("tesla_reddit_data3.csv")

def findstate(post_sub):
    sub_name = str(post_sub).lower()
    if sub_name in states:
        return sub_name
    elif sub_name in cities_to_state:
            return cities_to_state[sub_name]
    else:
        return "unkown"
        
def getpost(subred, find, time):
    subreddit = reddit.subreddit(subred)
    unique_posts_id = set()
    posts = []
    for post in subreddit.search(find, time_filter=time):
        # print(subred)
        if post.id not in unique_posts_id:
            unique_posts_id.add(post.id)
            posts.append({
                'id': post.id,
                'title': post.title,
                'selftext': post.selftext,
                'subreddit': post.subreddit.display_name,
                'url': post.url,
                'created_utc': post.created_utc,
                'upvotes': post.score,
                'comments': post.num_comments,
                'state': findstate(post.subreddit.display_name)
            })
    return pd.DataFrame(posts)

def get_state_post(time):
    new_tesla = pd.DataFrame([])
    states_col = []
    for state in states:
        post = getpost(state, 'tesla', time)
        if post.shape[0] !=0:
            new_tesla = pd.concat([new_tesla, post], axis = 0)

    new_tesla['created_utc'] = new_tesla['created_utc'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    return new_tesla

def get_city_post(time):
    new_tesla = pd.DataFrame([])
    states_col = []
    for city in cities_subbredit:
        print(city, end ='\t')
        post = getpost(city, 'tesla', time)
        if post.shape[0] !=0:
            new_tesla = pd.concat([new_tesla, post], axis = 0)

    new_tesla['created_utc'] = new_tesla['created_utc'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    return new_tesla



# Scratch work below

get_state_post() and get_cities_post() just need in a time variable to run: 'all', 'year','month', 'week' or 'day'

In [44]:
new_post_state = get_state_post('week')

C:\Users\olivi\AppData\Local\Temp\ipykernel_39812\4143237611.py:43: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  new_tesla['created_utc'] = new_tesla['created_utc'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))


In [46]:
new_post_state = get_city_post('week')

birmingham	fairbanks	phoenix	tucson	littlerock	fayetteville	losangeles	sandiego	santarosa	santabarbara	sanfrancisco	palmsprings	coloradosprings	aurora	hartford	miami	orlando	jacksonville	atlanta	augusta	roswell	honolulu	boise	nampa	naperville	chicago	indianapolis	bloomington	iowacity	desmoines	wichita	kansascity	louisville	lexington	neworleans	shreveport	frederickmd	annapolis	boston	salem	annarbor	grandrapids	detroit	ferndale	minneapolis	duluth	jacksonmi	tupelo	bransonmo	springfield	bozeman	helena	omaha	kearney	lasvegas	sparksnv	reno	HendersonNV	portsmouthnh	manchesternh	southjersey	newark	jerseycity	roswellnm	albuquerque	nyc	pittsburgh	buffalo	columbia	albany	charlotte	wilmington	greenvillencarolina	fargo	bismarck	minot	columbus	cleveland	toledo	cincinnati	dayton	akron	tulsa	okc	portland	portlandor	bend	eugene	philadelphia	philly	harrisburg	erie	charleston	myrtlebeach	greenville	siouxFalls	pierre	nashville	memphis	knoxville	houston	dallas	sanantonio	austin	elpaso	fortworth	lubbock	bed

C:\Users\olivi\AppData\Local\Temp\ipykernel_39812\4143237611.py:55: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  new_tesla['created_utc'] = new_tesla['created_utc'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))


In [65]:
post_to_save = pd.DataFrame()

In [50]:
post_to_save = pd.concat([post_to_save, new_post_state], axis = 0)
post_to_save = post_to_save.drop_duplicates(subset=['id'])
post_to_save.shape

(4369, 9)

## For add new data to CSV
Be careful not to erase old data

In [52]:
saved_post= pd.read_csv('tesla_only_geo_data.csv')

In [54]:
post_to_save = pd.concat([post_to_save, saved_post], axis = 0)
post_to_save = post_to_save.drop_duplicates(subset=['id'])
post_to_save.shape

(6207, 9)

In [60]:
post_to_save.to_csv('tesla_only_geo_data.csv',index=False)

In [224]:
#df.to_csv('file_name.csv',index=False)